In [1]:
from google.colab import drive
/drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

In [4]:
!tar xf spark-3.1.2-bin-hadoop3.2.tgz

In [5]:
!pip install -q findspark

In [13]:
!pip3 install pyspark

     |████████████████████████████████| 212.4 MB 54 kB/s 
  Using cached py4j-0.10.9-py2.py3-none-any.whl (198 kB)
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=5f3a24127737eda893f5674027c7f2b0d7e225199bc8bc9eb963be877e563585
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [15]:
import findspark
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as fn
from pyspark.sql import functions as sf

findspark.init()

In [ ]:
'''from pyspark import SparkContext
sc = SparkContext()'''

In [16]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [17]:
spark

In [18]:
df = spark.read.csv("/content/drive/MyDrive/german_data.csv", header=True, inferSchema=True)

In [19]:
df.show()

+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+
|Existing_account|Duration_month|Credit_history|Purpose|Credit_amount|Saving|Employment_duration|Installment_rate|Personal_status|Debtors|Residential_Duration|Property|Age|Installment_plans|Housing|Number_of_credits| Job|Liable_People|Telephone|Foreign_worker|Classification|
+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+
|             B11|             6|           A34|    A43|         1169|   A65|                A75|               4|            A93|   A101|                   4|    A121| 67|

In [20]:
df = df.filter((df.Purpose != 'NULL') & (df.Existing_account != 'NULL') & (df.Property !=  'NULL') & (df.Personal_status != 'NULL') & (df.Existing_account != 'NULL')  & (df.Credit_amount != 'NULL' ) & (df.Installment_plans != 'NULL'))

In [21]:
df.show()

+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+
|Existing_account|Duration_month|Credit_history|Purpose|Credit_amount|Saving|Employment_duration|Installment_rate|Personal_status|Debtors|Residential_Duration|Property|Age|Installment_plans|Housing|Number_of_credits| Job|Liable_People|Telephone|Foreign_worker|Classification|
+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+
|             B11|             6|           A34|    A43|         1169|   A65|                A75|               4|            A93|   A101|                   4|    A121| 67|

In [22]:
df = df.withColumn("Credit_amount", df['Credit_amount'].cast('float'))

In [23]:
df.show()

+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+
|Existing_account|Duration_month|Credit_history|Purpose|Credit_amount|Saving|Employment_duration|Installment_rate|Personal_status|Debtors|Residential_Duration|Property|Age|Installment_plans|Housing|Number_of_credits| Job|Liable_People|Telephone|Foreign_worker|Classification|
+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+
|             B11|             6|           A34|    A43|       1169.0|   A65|                A75|               4|            A93|   A101|                   4|    A121| 67|

In [24]:
split_col= pyspark.sql.functions.split(df['Existing_account'], '')
df = df.withColumn('Month', split_col.getItem(0))
df = df.withColumn('day1', split_col.getItem(1))
df = df.withColumn('day2', split_col.getItem(2))

df = df.withColumn('Days', sf.concat(sf.col('day1'),sf.col('day2')))
df = df.drop('day1')
df = df.drop('day2')
df.show()

+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+-----+----+
|Existing_account|Duration_month|Credit_history|Purpose|Credit_amount|Saving|Employment_duration|Installment_rate|Personal_status|Debtors|Residential_Duration|Property|Age|Installment_plans|Housing|Number_of_credits| Job|Liable_People|Telephone|Foreign_worker|Classification|Month|Days|
+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+-----+----+
|             B11|             6|           A34|    A43|       1169.0|   A65|                A75|               4|            A93|   A101| 

In [25]:
Month_Dict = {
    'A':'January',
    'B':'February',
    'C':'March',
    'D':'April',
    'E':'May',
    'F':'June',
    'G':'July',
    'H':'August',
    'I':'September',
    'J':'October',
    'K':'November',
    'L':'December'
    }
df = df.replace(Month_Dict,subset=['Month'])

In [26]:
df.show()

+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+---------+----+
|Existing_account|Duration_month|Credit_history|Purpose|Credit_amount|Saving|Employment_duration|Installment_rate|Personal_status|Debtors|Residential_Duration|Property|Age|Installment_plans|Housing|Number_of_credits| Job|Liable_People|Telephone|Foreign_worker|Classification|    Month|Days|
+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+---------+----+
|             B11|             6|           A34|    A43|       1169.0|   A65|                A75|               4|            A

In [27]:
split_purpose= pyspark.sql.functions.split(df['Purpose'], '')
df = df.withColumn('File_month', split_purpose.getItem(0))
df = df.withColumn('ver1', split_purpose.getItem(1))
df = df.withColumn('ver2', split_purpose.getItem(2))

df=df.withColumn('Version', sf.concat(sf.col('ver1'),sf.col('ver2')))
df = df.drop('ver1')
df = df.drop('ver2')
df.show()

+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+---------+----+----------+-------+
|Existing_account|Duration_month|Credit_history|Purpose|Credit_amount|Saving|Employment_duration|Installment_rate|Personal_status|Debtors|Residential_Duration|Property|Age|Installment_plans|Housing|Number_of_credits| Job|Liable_People|Telephone|Foreign_worker|Classification|    Month|Days|File_month|Version|
+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+---------+----+----------+-------+
|             B11|             6|           A34|    A43|       1169.0|

In [28]:
df= df.replace(Month_Dict,subset=['File_month'])

In [29]:
df.show()

+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+---------+----+----------+-------+
|Existing_account|Duration_month|Credit_history|Purpose|Credit_amount|Saving|Employment_duration|Installment_rate|Personal_status|Debtors|Residential_Duration|Property|Age|Installment_plans|Housing|Number_of_credits| Job|Liable_People|Telephone|Foreign_worker|Classification|    Month|Days|File_month|Version|
+----------------+--------------+--------------+-------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+---------+----+----------+-------+
|             B11|             6|           A34|    A43|       1169.0|

In [30]:
df = df.drop('Purpose')
df = df.drop('Existing_account')

In [31]:
df.show()

+--------------+--------------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+---------+----+----------+-------+
|Duration_month|Credit_history|Credit_amount|Saving|Employment_duration|Installment_rate|Personal_status|Debtors|Residential_Duration|Property|Age|Installment_plans|Housing|Number_of_credits| Job|Liable_People|Telephone|Foreign_worker|Classification|    Month|Days|File_month|Version|
+--------------+--------------+-------------+------+-------------------+----------------+---------------+-------+--------------------+--------+---+-----------------+-------+-----------------+----+-------------+---------+--------------+--------------+---------+----+----------+-------+
|             6|           A34|       1169.0|   A65|                A75|               4|            A93|   A101|                   4|    A121| 6

In [ ]:
#df = spark.read.csv("/content/drive/MyDrive/products.csv", header=True, inferSchema=True)

In [ ]:
mk = sc.parallelize([1,2,3,4])		
mk.take(1)
squared = mk.map(lambda x: x*x).collect()
squared
s = mk.count()
s

4

In [ ]:
#swimmers = spark.read.json('/content/drive/MyDrive/X_test_small.json')
swimmers = spark.read.format("json").load("/content/drive/MyDrive/testing.json")

In [ ]:
#df = spark.read.csv("/content/drive/MyDrive/products.csv", header=True, inferSchema=True)

In [ ]:
swimmers.createOrReplaceTempView("swimmers")

In [ ]:
spark.sql("Select * from swimmers").collect()

[Row(City='PARC PARQUE', Country='US', Decommisioned=False, EstimatedPopulation=None, Lat=17.96, Location='NA-US-PR-PARC PARQUE', LocationText='Parc Parque, PR', LocationType='NOT ACCEPTABLE', Long=-66.22, Notes=None, RecordNumber=1, State='PR', TaxReturnsFiled=None, TotalWages=None, WorldRegion='NA', Xaxis=0.38, Yaxis=-0.87, Zaxis=0.3, ZipCodeType='STANDARD', Zipcode=704),
 Row(City='PASEO COSTA DEL SUR', Country='US', Decommisioned=False, EstimatedPopulation=None, Lat=17.96, Location='NA-US-PR-PASEO COSTA DEL SUR', LocationText='Paseo Costa Del Sur, PR', LocationType='NOT ACCEPTABLE', Long=-66.22, Notes=None, RecordNumber=2, State='PR', TaxReturnsFiled=None, TotalWages=None, WorldRegion='NA', Xaxis=0.38, Yaxis=-0.87, Zaxis=0.3, ZipCodeType='STANDARD', Zipcode=704),
 Row(City='BDA SAN LUIS', Country='US', Decommisioned=False, EstimatedPopulation=None, Lat=18.14, Location='NA-US-PR-BDA SAN LUIS', LocationText='Bda San Luis, PR', LocationType='NOT ACCEPTABLE', Long=-66.26, Notes=None, R

In [ ]:
swimmers.select('City', 'Country').filter("City Like 'A%'").show()

+--------+-------+
|    City|Country|
+--------+-------+
|ASH HILL|     US|
|ASHEBORO|     US|
|ASHEBORO|     US|
+--------+-------+



In [ ]:
spark.sql('Select City Country from swimmers where City Like "A%"').show()

+--------+
| Country|
+--------+
|ASH HILL|
|ASHEBORO|
|ASHEBORO|
+--------+



In [ ]:
df = spark.createDataFrame([
(1, 144.5, 5.9, 33, 'M'),
(2, 167.2, 5.4, 45, 'M'),
(3, 124.1, 5.2, 23, 'F'),
(4, 144.5, 5.9, 33, 'M'),
(5, 133.2, 5.7, 54, 'F'),
(3, 124.1, 5.2, 23, 'F'),
(5, 129.2, 5.3, 42, 'M'),
], ['id', 'weight', 'height', 'age', 'gender'])

In [ ]:
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [ ]:
df = df.dropDuplicates()

In [ ]:
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  5| 129.2|   5.3| 42|     M|
|  1| 144.5|   5.9| 33|     M|
|  4| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



In [ ]:
airports = spark.read.csv("/content/drive/MyDrive/Testing.txt",header='true',
inferSchema='true', sep='\t')

In [ ]:
airports.createOrReplaceTempView("Airports")

In [ ]:
flightPerf = spark.read.csv("/content/drive/MyDrive/Testing2.csv",header='true')

In [ ]:

flightPerf.createOrReplaceTempView("FlightPerformance")

In [ ]:
flightPerf.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
|01061215|   -6|     602|   ABE|        ATL|
|01061725|   69|     602|   ABE|        ATL|
|01061230|    0|     369|   ABE|        DTW|
|01060625|   -3|     602|   ABE|        ATL|
|01070600|    0|     369|   ABE|        DTW|
|01071725|    0|     602|   ABE|        ATL|
|01071230|    0|     369|   ABE|        DTW|
|01070625|    0|     602|   ABE|        ATL|
|01071219|    0|     569|   ABE|        ORD|
|01080600|

In [ ]:
#header = flightPerf.first()
flightPerf.select('date').rdd.flatMap(list).collect()

In [ ]:
flightPerf.groupBy('distance').count().show()

+--------+-----+
|distance|count|
+--------+-----+
|     296| 1191|
|     829| 3898|
|     675|  362|
|    2069|  335|
|    1159|  553|
|     691|  337|
|    2294|  540|
|    1090|    1|
|     125| 1595|
|     451|  360|
|     944|  669|
|     800|  444|
|    1241| 1192|
|     666|  360|
|     124|  288|
|     447| 2351|
|     591| 3383|
|    1953| 1834|
|     475| 4908|
|     574|  407|
+--------+-----+
only showing top 20 rows



In [ ]:
titanic_df = spark.read.csv("/content/drive/MyDrive/titanic.csv",header='true')
titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [ ]:
titanic_df.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       0|  549|
|       1|  342|
+--------+-----+



In [ ]:
titanic_df.describe('Age').show()

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|               714|
|   mean| 29.69911764705882|
| stddev|14.526497332334035|
|    min|              0.42|
|    max|                 9|
+-------+------------------+



In [ ]:
titanic_df.filter(titanic_df.Cabin.isNull()).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|          330877| 8.4583| null|       Q|
|          8|       0|     3|Palsson, Master. ...|  male|   2|    3|    1|          349909| 21.075| null|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|  27|    0|    2|      

In [ ]:
titanic_df.filter(titanic_df.Cabin.isNull()).count()

687

In [ ]:
df = titanic_df.drop("Cabin")

In [ ]:
df.show()


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|          330877| 8.4583|       Q|
|          7|      

In [ ]:
df_new = df.filter(df.Age.isNotNull()).collect()

In [ ]:
df_new = spark.createDataFrame(df_new)

In [ ]:
df_new.show()

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male| 54|    0|    0|           17463|51.8625|       S|
|          8|       0|     3

In [ ]:
means = df_new.agg(*[fn.mean(c).alias(c) for c in df_new.columns if c == 'Age']).toPandas().to_dict('records')[0]

In [ ]:
means

{'Age': 29.69911764705882}

In [ ]:
#df.fillna(value = df.agg({"Age": 'mean'}), subset=["Age"])
df = df.fillna(value = means, subset=["Age"])

In [ ]:
df.show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Embarked|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|               22|    1|    0|       A/5 21171|   7.25|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|               38|    1|    0|        PC 17599|71.2833|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|               26|    0|    0|STON/O2. 3101282|  7.925|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|               35|    1|    0|          113803|   53.1|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|               35|    0|    0|          373450|   8.05|       S|
|          6|       0|  

In [ ]:
columns = ['Age','Fare']

In [ ]:
df_new = df_new.withColumn("Age", df_new['Age'].cast('float'))
df_new = df_new.withColumn("Fare", df_new['Fare'].cast('float'))

In [ ]:
df_new.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|           17463|51.8625|       S|
|          8|      

In [ ]:
bounds = {}
for col in columns:
  quantile = df_new.approxQuantile(col,[0.50, 0.75], 0.05)
  IQR = quantile[1] - quantile [0]
  bounds[col] = quantile[0] - 1.5 * IQR, quantile[1] + 1.5 * IQR

In [ ]:
bounds

{'Age': (16.0, 48.0), 'Fare': (-8.75, 53.25)}

In [ ]:
df_final = df_new[(df_new['Age'] >= bounds['Age'][0]) & (df_new['Age'] <= bounds['Age'][1]) & (df_new['Fare'] >= bounds['Fare'][0]) & (df_new['Fare'] <= bounds['Fare'][1])]


In [ ]:
df_final.count()

462

In [ ]:
!pip3 install bokeh

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
#import bokeh.plotting as chrt
#from bokeh.io import output_notebook
#output_notebook()

In [ ]:
spark.sql("""
Select a.City, 
f.origin
from FlightPerformance f 
join airports a 
on  a.IATA = f.origin 
where  a.State = "WA"
group by a.City, f.origin 
"""
).show()

In [ ]:
spark.sql("""
Select f.origin
from FlightPerformance  f 
where f.origin = 'ABE'
"""
).show()

+------+
|origin|
+------+
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
|   ABE|
+------+
only showing top 20 rows



In [ ]:
spark.sql("""
Select IATA
from Airports  
where IATA = 'ABE'
"""
).show()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



AnalysisException: ignored

In [ ]:
airports.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- IATA: string (nullable = true)



In [ ]:
%sql
Select a.City, f.origin, sum(f.delay) as Delays
from FlightPerformance f
join airports a
on a.IATA = f.origin
where a.State = 'WA'
group by a.City, f.origin
order by sum(f.delay) desc

In [ ]:
df_miss = spark.createDataFrame([ (1, 143.5, 5.6, 28,'M', 100000),
(2, 167.2, 5.4, 45, 'M', None),
(3, None , 5.2, None, None, None),
(4, 144.5, 5.9, 33, 'M', None),
(5, 133.2, 5.7, 54, 'F', None),
(6, 124.1, 5.2, None, 'F', None),
(7, 129.2, 5.3, 42, 'M', 76000),
], ['id', 'weight', 'height', 'age', 'gender', 'income'])

In [ ]:
df_miss.show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  1| 143.5|   5.6|  28|     M|100000|
|  2| 167.2|   5.4|  45|     M|  null|
|  3|  null|   5.2|null|  null|  null|
|  4| 144.5|   5.9|  33|     M|  null|
|  5| 133.2|   5.7|  54|     F|  null|
|  6| 124.1|   5.2|null|     F|  null|
|  7| 129.2|   5.3|  42|     M| 76000|
+---+------+------+----+------+------+



In [ ]:
df_miss.rdd.map(lambda row: (row['id'], sum([c == None for c in row]))).collect()

[(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (6, 2), (7, 0)]

In [ ]:
df_miss.rdd.map(lambda row: (row['id'], sum([c == None for c in row]))).collect()

[(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (6, 2), (7, 0)]

In [ ]:
df_miss.where('id == 3').show()
df_miss_no_income = df_miss.select([c for c in df_miss.columns if c != 'income'])
df_miss_no_income.dropna(thresh=3).show()
import pyspark.sql.types as typ